#Predict annotated trees in images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)
folder_path = '/content/drive/MyDrive/Thesis Johanne/Code'
import sys
sys.path.insert(0, folder_path)

Mounted at /content/drive


In [ ]:
# Install dependencies
!pip uninstall -y torchtext torchdata
!pip install torch==1.11.0+cu115 torchvision==0.12.0+cu115 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu115
!pip install -U openmim
!mim install mmengine
!mim install https://github.com/open-mmlab/mmdetection/archive/refs/tags/v2.28.2.zip

Found existing installation: torchtext 0.15.2
Uninstalling torchtext-0.15.2:
  Successfully uninstalled torchtext-0.15.2
Found existing installation: torchdata 0.6.1
Uninstalling torchdata-0.6.1:
  Successfully uninstalled torchdata-0.6.1
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu115
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 GB 437.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 7.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 

Looking in links: https://download.openmmlab.com/mmcv/dist/cu115/torch1.11.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.5/437.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 10.8 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu115/torch1.11.0/index.html
     / 20.8 MB 22.6 MB/s 0:00:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 MB 6.2 MB/s eta 0:00:00
  Created wheel for mmdet: filename=mmdet-2.28.2-py3-none-any.whl size=1494334 sha256=aadcf97039491c13368a23c813c34d8df44ab9ef56a2a5397f9e1ed2433a420d
  Stored in directory: /tmp/pip-ephem-wheel-cache-xvz5mls_/wheels/c3/fe/d6/76390a3734deeeeb6885c4ff3ebc40cea36128e06fd36ac20e
Successfully built mmdet


In [ ]:
import numpy as np
np.set_printoptions(formatter={'all':lambda x: str(x)})
import os
import pandas as pd
import json
import mmdet
import mmcv
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import inference_detector
from mmcv import Config
from mmdet.apis import inference_detector, init_detector, set_random_seed
from pycocotools.coco import COCO

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [ ]:
# Set random seed
seed = 3047
set_random_seed(seed, deterministic=False)

In [ ]:
def predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id):
    # Load ground truth annotations
    with open(filepath_gt, 'r') as fp:
        anns_gt = json.load(fp)
    n_imgs = len(anns_gt['images'])
    print(f'\tN. of images:', n_imgs)

    # Make predictions
    anns_pred = anns_gt.copy()
    anns_pred['annotations'] = []
    for i, img in enumerate(anns_gt['images']):
        im = mmcv.imread(f'{filepath_imgs}/{img["file_name"]}')
        preds = inference_detector(model, im)
        for p in preds[0]:
            x1, y1, x2, y2 = p[:-1]
            bbox = [x1, y1, x2-x1, y2-y1]
            bbox = [float(elem) for elem in bbox]
            anns_pred['annotations'].append({'id': ann_id,
                                            'image_id': img['id'],
                                            'category_id': 0,
                                            'bbox': bbox,
                                            'score': float(p[-1])})
            ann_id += 1

        # Print progress
        if (i > 0) & (i % 250 == 0):
            print(f'\t\t{round((i / n_imgs) * 100)}%')

    # Save predicted annotations
    with open(filepath_pred, 'w') as fp:
        json.dump(anns_pred, fp)

    return ann_id

In [ ]:
# Initialize the model
config = Config.fromfile(f'{folder_path}/config.py')
checkpoint = f'{folder_path}/model_output_4/model.pth'
model = init_detector(config, checkpoint, device='cuda:0');

### Make predictions on Copenhagen dataset

In [ ]:
# Make predictions for each split
filepath_imgs = f'{folder_path}/data/streetviews'
ann_id = 0
for split in ['train', 'val', 'test']:
    print(f'{split.capitalize()}:')
    filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label.json'
    filepath_pred = f'{folder_path}/model_output/predictions_annotations_{split}.json'
    ann_id = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id)

load checkpoint from local path: /content/drive/MyDrive/Thesis Johanne/Code/model_output_4/best_bbox_mAP_epoch_5.pth
Train:
loading annotations into memory...
Done (t=0.81s)
creating index...
index created!
	N. of images: 8122
		3%
		6%
		9%
		12%
		15%
		18%
		22%
		25%
		28%
		31%
		34%
		37%
		40%
		43%
		46%
		49%
		52%
		55%
		58%
		62%
		65%
		68%
		71%
		74%
		77%
		80%
		83%
		86%
		89%
		92%
		95%
		98%
Val:
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
	N. of images: 2932
		9%
		17%
		26%
		34%
		43%
		51%
		60%
		68%
		77%
		85%
		94%
Test:
loading annotations into memory...
Done (t=0.39s)
creating index...
index created!
	N. of images: 2637
		9%
		19%
		28%
		38%
		47%
		57%
		66%
		76%
		85%
		95%


###Make predictions on Pasadena dataset

In [ ]:
# Make predictions for Pasadena dataset using original model
split = 'test'
print(f'{split.capitalize()}:')
filepath_imgs = f'{folder_path}/data/images/pasadena_2'
filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label_pd.json'
filepath_pred = f'{folder_path}/model_output/predictions_annotations_{split}_pd.json'
_ = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id=0)

Test:
	N. of images: 2262
		11%
		22%
		33%
		44%
		55%
		66%
		77%
		88%
		99%


In [ ]:
# Initialize the finetuned model
config = Config.fromfile(f'{folder_path}/config.py')
checkpoint = f'{folder_path}/model_output_6/model_finetuned.pth'
model = init_detector(config, checkpoint, device='cuda:0');

In [ ]:
# Make predictions on Pasadena dataset using fine-tuned model
split = 'test'
print(f'{split.capitalize()}:')
filepath_imgs = f'{folder_path}/data/images/pasadena_2'
filepath_gt = f'{folder_path}/data/annotations/annotations_{split}_single_label_pd.json'
filepath_pred = f'{folder_path}/model_output/predictions_annotations_{split}_pd_ft.json'
_ = predict_annotations(model, filepath_gt, filepath_pred, filepath_imgs, ann_id=0)